In [1]:
import pandas as pd
import numpy as np

In [2]:
# 读取数据集
df1 = pd.read_excel('./示例文件.xlsx',sheet_name='data1')
df1

,orderID,product,category,userID,monetary,date
0,O1,木桌,家具,A,400,2021-01-01
1,O2,薯片,食品,B,10,2021-01-01
2,O3,椅子,家具,A,100,2021-01-02
3,O4,毛衣,服装,C,200,2021-01-02
4,O5,面包,食品,A,25,2021-01-03
5,O6,牛仔裤,服装,B,150,2021-01-03
6,O7,饮用水,食品,D,60,2021-01-04


In [3]:
df2 = pd.read_excel('./示例文件.xlsx',sheet_name='data2')
df2

,userID,reg_date
0,A,2021-01-01
1,B,2021-01-01
2,C,2021-01-02
3,D,2021-01-02


In [4]:
# 筛选出各用户消费的总金额
df = df1.groupby(['userID'])['monetary'].sum().reset_index()
df

,userID,monetary
0,A,525
1,B,160
2,C,200
3,D,60


In [5]:
# 筛选出各品类消费金额最多的订单
df1['组内降序排名'] = df1.groupby(['category'])['monetary'].rank(method='dense',ascending=False).astype(int)
df = df1[df1['组内降序排名'] == 1]
df = df[['category','orderID']]
df

,category,orderID
0,家具,O1
3,服装,O4
6,食品,O7


In [6]:
# 筛选出老用户（注册日当日未新用户）的平均购买金额
df_merge = pd.merge(df1,df2,on='userID',how='left')
df_merge['用户分类'] = np.where(df_merge['date'] == df_merge['reg_date'],'新用户','老用户')
df_merge = df_merge[df_merge['用户分类'] == '老用户']
avg_monetary = df_merge['monetary'].mean()
avg_monetary

83.75

In [7]:
# 筛选出连续 3 日有购买记录的用户
df = df1[['date','userID']].drop_duplicates()
df['rank'] = df['date'].groupby(df['userID']).rank(method='dense',ascending=True).astype(int)
df = df.sort_values(by=['userID','date'])
df['days'] = df['rank'].map(lambda x:pd.Timedelta(days=x))
df['minus'] = df['date'] - df['days']
df_group1 = df.groupby(['userID','minus'])['rank'].max().reset_index()
df_group2 = df_group1.groupby('userID')['rank'].max().reset_index()
df = df_group2[df_group2['rank'] == 3]
df

,userID,rank
0,A,3
